<table width = "100%">
  <tr style="background-color:white;">
    <!-- QWorld Logo -->
    <td style="text-align:left;width:200px;"> 
        <a href="https://qworld.net/" target="_blank"><img src="../images/QWorld.png"> </a></td>
    <td style="text-align:right;vertical-align:bottom;font-size:16px;"> 
        Prepared by <a href="https://www.cmpe.boun.edu.tr/~ozlem.salehi/" target="_blank"> Özlem Salehi </a> </td>
    </tr> 
 </table>
 
<hr>

# Quantum Annealing on D-Wave

Finally, we have reached to the point where we will access D-Wave devices and run our codes on real machines instead of using simulated annealing. 

Let's start with a very simple example.

You can observe that instead of `SimulatedAnnealingSampler`, we create an instance of `DWaveSampler`. Functions for obtaining the sample, i.e. `sample`, `sample_qubo`, `sample_ising` are the same for both samplers.

In [13]:
from dimod import BQM

from dwave.system import DWaveSampler
sampler = DWaveSampler()

quadratic = {('s_1', 's_2'): 4}
vartype = 'SPIN'

bqm = BQM(quadratic, vartype)

sampleset = sampler.sample(bqm, num_reads=10)
print(sampleset)

BinaryQuadraticModelStructureError: Problem graph incompatible with solver. Please use 'EmbeddingComposite' to map the problem graph to the solver.

However, when you run the code you will encounter an error

> BinaryQuadraticModelStructureError: Problem graph incompatible with solver. Please use 'EmbeddingComposite' to map the problem graph to the solver.


The problem is that there are no qubits named `x1` and `x2` on the real device. 

Let's get the node list of the real device:

In [10]:
sampler.nodelist

[30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185,
 186,
 187,
 188,
 189,
 190,
 191,
 192,
 193,
 194,
 195,
 196,
 197,
 198,
 199,
 200,
 201,
 202,
 203,
 204,
 205,
 206,
 207,
 2

This time let's input 30 and 31 as our qubit names.

In [11]:
from dimod import BQM

from dwave.system import DWaveSampler
sampler = DWaveSampler()

quadratic = {(30, 31): 4}
vartype = 'SPIN'

bqm = BQM(quadratic, vartype)

sampleset = sampler.sample(bqm, num_reads=10)
print(sampleset)

  30 31 energy num_oc.
0 -1 +1   -4.0       9
1 +1 -1   -4.0       1
['SPIN', 2 rows, 10 samples, 2 variables]


You see that it worked! Let's try another one.

In [14]:
from dimod import BQM

from dwave.system import DWaveSampler
sampler = DWaveSampler()

quadratic = {(30, 31): 4, (30, 32):4, (31, 32):4}
vartype = 'SPIN'

bqm = BQM(quadratic, vartype)

sampleset = sampler.sample(bqm, num_reads=10)
print(sampleset)

BinaryQuadraticModelStructureError: Problem graph incompatible with solver. Please use 'EmbeddingComposite' to map the problem graph to the solver.

We again encoounter an error. What is the reason?

Note that not all the qubits are connected to each other. You may try to get the adjacency list for the D-Wave machine and encode your qubits accordingly. However, there exist no three qubits which are mutually connected. 

<img src="../images/chimera.png" width="200">

Yet, the bqm we have created requires such connection to be implemented.

The solution is known as **minor embedding**.

## Minor Embedding

When we define a binary quadratic model, the variables we use are called *logical qubits*. Logical qubits needs to be mapped to the *physical qubits* in the actual device. This process is known as the *minor embedding*.

<img src="../images/minor1.png" width="200">


Two-variable problem, shown on the left as a graph, is embedded in two connected qubits on a D-Wave 2000Q, shown on the right against the Chimera topology. Variable , highlighted in dark magenta, is represented by qubit number 1929 and variable  is represented by qubit 1801. (Taken from D-Wave)

### Chains

Not all connections are available on the real device. To overcome this, each logical qubit, may be represented by either one or more than one physical qubit on the actual device. 

<img src="../images/minor2.png" width="200">


For the Ising model $4s_1s_2+4s_2s_3+4s_1s_3$, we need to allocate 4 qubits in the actual device. The two qubits in pick represent the same qubit and they are called a *chain*.

The coupling between them is called the *chain strength*. The chain strength should be set high enough so the two qubits always get the same value and act if as they are a single qubit. On the other hand, too high chain strength may override the problem itself.

Now the question is how to perform minor embedding?

Ocean SDK provides a functionality so that you can automatically embed your problems to the device. This is how you should create your sampler. 

In [17]:
sampler = EmbeddingComposite(DWaveSampler())

NameError: name 'EmbeddingComposite' is not defined

In [ ]:
from dimod import BQM

from dwave.system import DWaveSampler
sampler = DWaveSampler()

linear = {'x1': -5, 'x2': -3, 'x3': -8, 'x4': -6}
quadratic = {('x1', 'x2'): 4, ('x1', 'x3'): 8, ('x2', 'x3'): 2, ('x3', 'x4'): 10}
vartype = 'BINARY'

bqm = BQM(linear, quadratic, vartype)

sampleset = sampler.sample(bqm, num_reads=10)
print(sampleset)